In [ ]:
import tensorflow as tf
import random
import numpy as np
from tensorflow import keras

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
h = w = 28
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

In [ ]:
def create_train_set(num):
    '''
    num: batch_size.
    
    '''
    train_batch_list = random.sample(range(len(X_train)),num)
    label_onehot = np.zeros((num,10))
    train_set = np.zeros((num,h,w))
    for i,item in enumerate(train_batch_list):
        train_set[i,:,:] = X_train[item]
        label_onehot[i][y_train[item]] = 1
    return train_set/255, label_onehot


In [ ]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

filter_1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
layer_1 = tf.nn.atrous_conv2d(value=X, filters=filter_1, rate=2, padding='SAME')
layer_1 = tf.nn.relu(layer_1)
layer_1 = tf.nn.max_pool(layer_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


filter_2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
layer_2 = tf.nn.atrous_conv2d(value=layer_1, filters=filter_2, rate=2, padding='SAME')
layer_2 = tf.nn.relu(layer_2)
layer_2 = tf.nn.max_pool(layer_2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

weight_fc = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
layer_fc = tf.reshape(layer_2, [-1, 7 * 7 * 64])
layer_fc = tf.matmul(layer_fc, weight_fc)
layer_fc = tf.nn.relu(layer_fc)

W = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(layer_fc, W)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.003).minimize(cost)
correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))


In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
tf.reset_default_graph()
batch_size = 128
total_batch = int(len(X_train) / batch_size)

for epoch in range(10):
    total_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = create_train_set(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs,
                                          Y: batch_ys,})
        total_cost += cost_val
        if i % 50 == 0:
            batch_xs_test, batch_ys_test = create_train_set(batch_size)
            batch_xs_test = batch_xs_test.reshape(-1, 28, 28, 1)
            print('train acc:', sess.run(accuracy,
                                    feed_dict={X: batch_xs_test,
                                               Y: batch_ys_test}))
            print('val acc:', sess.run(accuracy,
                                    feed_dict={X: X_test.reshape(-1, 28, 28, 1),
                                               Y: np.eye(10)[y_test],}))